# KNN Item-Item Collaborative Filtering

Item-Item CF is similar to User-User CF but works on the other side. Instead of finding similar users, this method identify the similar  items which the current user has interacted with, get the current user's ratings for them and weighted average the actual ratings.

Based on [Introduction to Recommender System](https://towardsdatascience.com/introduction-to-recommender-systems-6c66cf15ada):
> The user-user method is based on the search of similar users in terms of interactions with items. As, in general, every user have only interacted with a few items, it makes the method pretty sensitive to any recorded interactions (high variance). On the other hand, as the final recommendation is only based on interactions recorded for users similar to our user of interest, we obtain more personalized results (low bias).

> Conversely, the item-item method is based on the search of similar items in terms of user-item interactions. As, in general, a lot of users have interacted with an item, the neighbourhood search is far less sensitive to single interactions (lower variance). As a counterpart, interactions coming from every kind of users (even users very different from our reference user) are then considered in the recommendation, making the method less personalised (more biased). Thus, this approach is less personalized than the user-user approach but more robust.

In this notebook, we would take a very simple and straight-forward approach to implement KNN Item-Item CF. We formulate the problem as predicting the rating between user U and item I based on the rating records.

Specifically, we will:
- Collect input containing user-item rating
- Build item-item similarity matrix
- Measure similarities between items by using plain user rating vectors
- Predict the rating between user U and item I by getting all items which the current user have rated, identify N item neighbors and weighted average their ratings based on how similar the items are to the target item I

# Set up

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

import mlflow
import numpy as np
import pandas as pd
from dotenv import load_dotenv
from loguru import logger
from pydantic import BaseModel

load_dotenv()

sys.path.insert(0, "..")

from src.eval import (
    create_label_df,
    create_rec_df,
    log_classification_metrics,
    log_ranking_metrics,
    merge_recs_with_target,
)
from src.id_mapper import IDMapper
from src.math_utils import sigmoid
from src.model import Item2ItemCollaborativeFiltering
from src.train_utils import map_indice
from src.viz import blueq_colors

# Controller

In [3]:
class Args(BaseModel):
    testing: bool = False
    log_to_mlflow: bool = True
    experiment_name: str = "FSDS RecSys - L5 - Reco Algo"
    run_name: str = "003-cf-i2i"
    notebook_persist_dp: str = None
    random_seed: int = 41

    user_col: str = "user_id"
    item_col: str = "parent_asin"
    rating_col: str = "rating"
    timestamp_col: str = "timestamp"

    top_K: int = 100
    top_k: int = 10

    batch_size: int = 128

    def init(self):
        self.notebook_persist_dp = os.path.abspath(f"data/{self.run_name}")

        if not os.environ.get("MLFLOW_TRACKING_URI"):
            logger.warning(
                f"Environment variable MLFLOW_TRACKING_URI is not set. Setting self.log_to_mlflow to false."
            )
            self.log_to_mlflow = False

        if self.log_to_mlflow:
            logger.info(
                f"Setting up MLflow experiment {self.experiment_name} - run {self.run_name}..."
            )

            mlflow.set_experiment(self.experiment_name)
            mlflow.start_run(run_name=self.run_name)

        return self


args = Args().init()

print(args.model_dump_json(indent=2))

2024-09-22 16:12:47.576 | INFO     | __main__:init:29 - Setting up MLflow experiment FSDS RecSys - L5 - Reco Algo - run 003-cf-i2i...


{
  "testing": false,
  "log_to_mlflow": true,
  "experiment_name": "FSDS RecSys - L5 - Reco Algo",
  "run_name": "003-cf-i2i",
  "notebook_persist_dp": "/Users/dvq/frostmourne/reco-algo/notebooks/data/003-cf-i2i",
  "random_seed": 41,
  "user_col": "user_id",
  "item_col": "parent_asin",
  "rating_col": "rating",
  "timestamp_col": "timestamp",
  "top_K": 100,
  "top_k": 10,
  "batch_size": 128
}


# Implement

In [4]:
def init_model(n_users, n_items):
    model = Item2ItemCollaborativeFiltering(n_users, n_items)
    return model

# Test implementation

In [5]:
# Mock data
user_indices = [0, 0, 1, 1, 2, 2, 2]
item_indices = [0, 1, 1, 2, 3, 1, 2]
ratings = [1, 4, 4, 5, 3, 2, 4]
n_users = len(set(user_indices))
n_items = len(set(item_indices))

val_user_indices = [0, 1, 2]
val_item_indices = [2, 1, 2]
val_ratings = [2, 4, 5]

print("Mock User IDs:", user_indices)
print("Mock Item IDs:", item_indices)
print("Ratings:", ratings)

model = init_model(n_users, n_items)

users = [0, 1, 2]
items = [2, 2, 0]
predictions = model.predict(users, items)
print(predictions)

Mock User IDs: [0, 0, 1, 1, 2, 2, 2]
Mock Item IDs: [0, 1, 1, 2, 3, 1, 2]
Ratings: [1, 4, 4, 5, 3, 2, 4]
[0.5 0.5 0.5]


In [6]:
model.fit(user_indices, item_indices, ratings)
predictions = model.predict(users, items)
print(predictions)

[0.98201379 0.98201379 0.88079708]


#### 🧐 Go into details

In [7]:
model.user_item_matrix.T

array([[1., 0., 0.],
       [4., 4., 2.],
       [0., 5., 4.],
       [0., 0., 3.]])

In [8]:
model.item_similarity

array([[0.        , 0.66666667, 0.        , 0.        ],
       [0.66666667, 0.        , 0.72881089, 0.33333333],
       [0.        , 0.72881089, 0.        , 0.62469505],
       [0.        , 0.33333333, 0.62469505, 0.        ]])

In [9]:
item = 3
user = 1

# Compute prediction using weighted average of ratings from similar items
sim_scores = model.item_similarity[item]
print(f"{sim_scores=}")

sim_scores=array([0.        , 0.33333333, 0.62469505, 0.        ])


In [10]:
# Only consider items that have been rated by the current user
item_ratings = model.user_item_matrix[user, :]
print(f"Ratings of current user for all items:\n{item_ratings=}")
sim_scores = sim_scores[item_ratings != 0]
print(
    f"Cosine similarity score of target item towards all other items where current user has rated:\n{sim_scores}"
)
item_ratings = item_ratings[item_ratings != 0]

Ratings of current user for all items:
item_ratings=array([0., 4., 5., 0.])
Cosine similarity score of target item towards all other items where current user has rated:
[0.33333333 0.62469505]


In [11]:
# Weighted average of ratings
print(f"Weighted average: {np.dot(sim_scores, item_ratings)}")
print(f"Normalization factor: {np.sum(sim_scores)}")
print(f"Predicted rating: {np.dot(sim_scores, item_ratings) / np.sum(sim_scores)}")
print(
    f"Predicted rating - sigmoid: {sigmoid(np.dot(sim_scores, item_ratings) / np.sum(sim_scores))}"
)

Weighted average: 4.456808571105455
Normalization factor: 0.9580283808877577
Predicted rating: 4.652063195638892
Predicted rating - sigmoid: 0.9905482923878774


In [12]:
recommendations = model.recommend(val_user_indices, k=2)

Generating Recommendations:   0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
recommendations

{'user_indice': [0, 0, 1, 1, 2],
 'recommendation': [2, 3, 3, 0, 0],
 'score': [0.9820137900379085,
  0.9820137900379085,
  0.9905482923878774,
  0.9820137900379085,
  0.8807970779778823]}

# Prep data

In [14]:
train_df = pd.read_parquet("../data/train_features_neg_df.parquet")
val_df = pd.read_parquet("../data/val_features_neg_df.parquet")
idm = IDMapper().load("../data/idm.json")
# val_timestamp = 1628643414042  # https://amazon-reviews-2023.github.io/data_processing/5core.html
assert (val_df[args.timestamp_col].min() - train_df[args.timestamp_col].max()) > 0
val_timestamp = train_df[args.timestamp_col].max() + 1
print(f"{val_timestamp=}")

val_timestamp=np.int64(1628641464793)


In [15]:
user_ids = train_df[args.user_col].values
item_ids = train_df[args.item_col].values
unique_user_ids = list(set(user_ids))
unique_item_ids = list(set(item_ids))
n_users = len(unique_user_ids)
n_items = len(unique_item_ids)

logger.info(f"{len(unique_user_ids)=:,.0f}, {len(unique_item_ids)=:,.0f}")

2024-09-22 16:12:48.954 | INFO     | __main__:<module>:8 - len(unique_user_ids)=20,366, len(unique_item_ids)=4,696


In [16]:
train_df = train_df.pipe(map_indice, idm, args.user_col, args.item_col)
val_df = val_df.pipe(map_indice, idm, args.user_col, args.item_col)

user_indices = [idm.get_user_index(user_id) for user_id in user_ids]
item_indices = [idm.get_item_index(item_id) for item_id in item_ids]
ratings = train_df[args.rating_col].values.tolist()

val_user_indices = [idm.get_user_index(user_id) for user_id in val_df[args.user_col]]
val_item_indices = [idm.get_item_index(item_id) for item_id in val_df[args.item_col]]
val_ratings = val_df[args.rating_col].values.tolist()

# Train

In [17]:
model = init_model(n_users, n_items)

#### Predict before train

In [18]:
user_id = val_df.sample(1)[args.user_col].values[0]
test_df = val_df.loc[lambda df: df[args.user_col].eq(user_id)]
test_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence
1085,AHRDCITUE5NPFLAZQG6JZ6FOQEPQ,B001EYUU2O,0.0,1638716160130,6200,6,Video Games,Need for Speed Carbon - Xbox 360,[What starts in the city is settled in the can...,"[Video Games, Legacy Systems, Xbox Systems, Xb...",39.96,"[-1, -1, -1, -1, -1, 806, 2562, 3690, 96, 3531]"
1880,AHRDCITUE5NPFLAZQG6JZ6FOQEPQ,B07796MBJ7,3.0,1638716160130,6200,3293,Computers,Logitech G613 LIGHTSPEED Wireless Mechanical G...,"[Designed for a wireless world, G613 is a high...","[Video Games, PC, Accessories, Gaming Keyboards]",72.99,"[-1, -1, -1, -1, -1, 806, 2562, 3690, 96, 3531]"


In [19]:
item_id = test_df.loc[lambda df: df[args.rating_col].gt(0)][args.item_col].values[0]
logger.info(
    f"Test predicting before training with {args.user_col} = {user_id} and {args.item_col} = {item_id}"
)
user_indice = idm.get_user_index(user_id)
item_indice = idm.get_item_index(item_id)

model.predict([user_indice], [item_indice])

2024-09-22 16:12:49.205 | INFO     | __main__:<module>:2 - Test predicting before training with user_id = AHRDCITUE5NPFLAZQG6JZ6FOQEPQ and parent_asin = B07796MBJ7


array([0.5])

#### Training loop

In [20]:
model.fit(user_indices, item_indices, ratings)

# Predict

In [21]:
logger.info(
    f"Test predicting before training with {args.user_col} = {user_id} and {args.item_col} = {item_id}"
)
model.predict([user_indice], [item_indice])

2024-09-22 16:12:51.015 | INFO     | __main__:<module>:1 - Test predicting before training with user_id = AHRDCITUE5NPFLAZQG6JZ6FOQEPQ and parent_asin = B07796MBJ7


array([0.5])

# Evaluate

## Ranking metrics

In [22]:
recommendations = model.recommend(val_user_indices, k=args.top_K)

Generating Recommendations:   0%|          | 0/1898 [00:00<?, ?it/s]

In [23]:
recommendations_df = pd.DataFrame(recommendations).pipe(create_rec_df, idm)
recommendations_df

,user_indice,recommendation,score,rec_ranking,user_id,parent_asin
0,11638,3462,0.993307,1.0,AHO3T4INZKH5C7IIV5JERA3H2SKQ,B004LLHFEI
1,11638,442,0.993307,2.0,AHO3T4INZKH5C7IIV5JERA3H2SKQ,B013JDH8J0
2,11638,1222,0.993307,3.0,AHO3T4INZKH5C7IIV5JERA3H2SKQ,B008MWIALE
3,11638,3953,0.993307,4.0,AHO3T4INZKH5C7IIV5JERA3H2SKQ,B073ST27MK
4,11638,3954,0.993307,5.0,AHO3T4INZKH5C7IIV5JERA3H2SKQ,B0050SVGXM
...,...,...,...,...,...,...
189795,10038,4200,0.993307,196.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B00JA81WQE
189796,10038,1169,0.993307,197.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B00KSY5486
189797,10038,3414,0.993307,198.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B06XS4XXFT
189798,10038,1655,0.993307,199.0,AHAKU6TTWIHJPZIODW7MGC52M2DA,B01NCVU39G


In [24]:
label_df = create_label_df(val_df)
label_df

,user_id,parent_asin,rating,rating_rank
548,AFI3SEGKSS7X7CVOC3HS5U5RAIWQ,B077GG9D5D,5.0,1.0
697,AGDFPA6XVENIY7XJPIOXBUHRJQ6A,B09V25XG1G,5.0,1.0
109,AHMJVCKVHJIT2R5NWWV4HG4TDH6A,B07C2XYDW8,5.0,1.0
197,AGQAPJGNR3IDAJCPD2YPU7CRRPBA,B0C7BN9G35,5.0,1.0
797,AF6IUPCJM4FDNOQZNCDOGTLM2M4Q,B09T5VN7D1,4.0,1.0
...,...,...,...,...
850,AFB6FYPPCN33UMUU5536IHXNOHCQ,B001E8WQJA,0.0,18.0
1106,AG4RCXKPTC6QRORJLUSBY4SO2IAA,B00DBLBMBQ,0.0,18.0
1003,AESD4RLWUKM6JTD6SNNWYLHLLQQA,B00Z9TMBOU,0.0,18.0
52,AFB6FYPPCN33UMUU5536IHXNOHCQ,B0892HRCST,0.0,19.0


In [25]:
eval_df = merge_recs_with_target(recommendations_df, label_df, k=args.top_K)
eval_df

,user_indice,recommendation,score,rec_ranking,user_id,parent_asin,rating,rating_rank
37,14192.0,2591.0,0.993307,1,AE2AZ2MNROPF33U6SS53VI22OXJA,B001EYUQKU,0,NaN
91,14192.0,549.0,0.993307,2,AE2AZ2MNROPF33U6SS53VI22OXJA,B008J35YFQ,0,NaN
171,14192.0,336.0,0.993307,3,AE2AZ2MNROPF33U6SS53VI22OXJA,B07V3T7WF1,0,NaN
95,14192.0,4557.0,0.993307,4,AE2AZ2MNROPF33U6SS53VI22OXJA,B00DC7G3Q8,0,NaN
169,14192.0,2080.0,0.993307,5,AE2AZ2MNROPF33U6SS53VI22OXJA,B07SV4RKZ7,0,NaN
...,...,...,...,...,...,...,...,...
191504,16101.0,4293.0,0.993307,196,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B003Y7RB76,0,NaN
191566,16101.0,3767.0,0.993307,197,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B00QLWZWMW,0,NaN
191461,16101.0,4283.0,0.993307,198,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B000NDFFFE,0,NaN
191530,16101.0,1517.0,0.993307,199,AHZNHP6OKXRZV2UJMYDPLWCKFKEA,B00DBRNRPE,0,NaN


In [26]:
ranking_report = log_ranking_metrics(args, eval_df)

/Users/dvq/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/evidently/metrics/recsys/f_beta_top_k.py:61: RuntimeWarning: invalid value encountered in divide
  return (1 + beta_sqr) * precision_arr * recall_arr / (beta_sqr * precision_arr + recall_arr)


## Classification metrics

In [27]:
val_user_indices = val_df["user_indice"].values
val_item_indices = val_df["item_indice"].values

In [28]:
classifications = model.predict(val_user_indices, val_item_indices)

In [29]:
eval_classification_df = val_df.assign(
    classification_proba=classifications,
    label=lambda df: df[args.rating_col].gt(0).astype(int),
)
eval_classification_df

,user_id,parent_asin,rating,timestamp,user_indice,item_indice,main_category,title,description,categories,price,item_sequence,classification_proba,label
0,AHO3T4INZKH5C7IIV5JERA3H2SKQ,B01GY35QPU,0.0,1638057548682,11638,1902,Video Games,Detroit Become Human - PlayStation 4,[Detroit: Become Human is the latest title in ...,"[Video Games, PlayStation 4, Games]",21.4,"[3579, 878, 434, 1579, 3669, 3322, 3495, 2339,...",0.500000,0
1,AEMYS2WV33NDV3OAJFOM7SSSIDTA,B0088TN7BO,0.0,1640914537652,14788,2180,Video Games,LEGO Lord of the Rings - Nintendo 3DS,"[Product Description, Based on The Lord of the...","[Video Games, Legacy Systems, Nintendo Systems...",19.95,"[-1, -1, -1, -1, -1, 3244, 2872, 2621, 3048, 3...",0.500000,0
2,AEXKGQQMYQQUNWVGD66TG3VT4V4A,B00IPTUJ8G,0.0,1653120417877,19789,4619,Video Games,Borderlands 2,"[Borderlands 2 for PS Vita, View Larger, View ...","[Video Games, Legacy Systems, PlayStation Syst...",48.1,"[-1, -1, -1, -1, -1, 2358, 3142, 4521, 2292, 2...",0.951095,0
3,AGQAPJGNR3IDAJCPD2YPU7CRRPBA,B00CMQTVK0,0.0,1643392366551,6414,1765,Video Games,Xbox One with Kinect (Day One Edition),"[Get more with Xbox One., Introducing Xbox One...","[Video Games, Xbox One, Consoles]",589.99,"[-1, -1, -1, -1, 3142, 4274, 4345, 809, 416, 1...",0.993307,0
4,AGIJWTPKBANKWNEM2AHK7PWTBYLQ,B01BF9X9VQ,0.0,1647513889874,1195,2412,Video Games,World of Warcraft: Legion - Standard Edition -...,"[Kingdoms will burn, The Burning Legion surges...","[Video Games, PC, Games]",28.6,"[170, 3609, 1588, 1467, 3786, 1627, 4234, 3056...",0.964872,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1893,AF5T2J7T33UFSPUGCKTPTGU7EY7A,B008I2LRMC,0.0,1654892101229,14749,1743,Video Games,F1: 2012,"[Product Description, F1 2012 is designed to b...","[Video Games, Legacy Systems, Xbox Systems, Xb...",33.03,"[601, 1842, 3391, 2640, 761, 3624, 1079, 2656,...",0.500000,0
1894,AGXQLR7TWHVUQLDBKKOBI4OQQGRQ,B01N3ASPNV,0.0,1637285457188,8208,3069,All Electronics,amFilm Tempered Glass Screen Protector for Nin...,[],"[Video Games, Nintendo Switch, Accessories, Fa...",8.91,"[-1, -1, 4227, 1039, 1352, 2619, 2655, 1305, 1...",0.988852,0
1895,AFH63KLSVQQYRNFS7NLQGD3GSP3A,B094YHB1QK,5.0,1652564728981,9221,2581,Video Games,PlayStation DualSense Wireless Controller – Ga...,[Plot a course for astronomical adventures on ...,"[Video Games, PlayStation 5, Accessories, Cont...",74.99,"[-1, 2705, 3691, 1804, 111, 3330, 1775, 2979, ...",0.993307,1
1896,AFPPTJOEUPVXA5C63SNRGID3EQNA,B0BVVTQ5JP,4.0,1635968491390,4173,3191,Computers,Logitech G502 HERO High Performance Wired Gami...,[Logitech updated its iconic G502 gaming mouse...,"[Video Games, PC, Accessories, Gaming Mice]",45.87,"[-1, -1, -1, -1, -1, 2379, 1799, 1932, 1568, 161]",0.973120,1


In [30]:
classification_report = log_classification_metrics(
    args,
    eval_classification_df,
    target_col="label",
    prediction_col="classification_proba",
)

/Users/dvq/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/dvq/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/Users/dvq/frostmourne/reco-algo/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



# Clean up

In [31]:
all_params = [args]

if args.log_to_mlflow:
    for params in all_params:
        params_dict = params.dict()
        params_ = {f"{params.__repr_name__()}.{k}": v for k, v in params_dict.items()}
        mlflow.log_params(params_)

    mlflow.end_run()

2024/09/22 16:12:55 INFO mlflow.tracking._tracking_service.client: 🏃 View run 003-cf-i2i at: http://localhost:5003/#/experiments/1/runs/0cc69d3a959a41a59b57cb3794704e5a.
2024/09/22 16:12:55 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://localhost:5003/#/experiments/1.


# Appendix

## Model returning same score for every user-item in top 100